In [1]:
import pandas as pd

## Wczytanie danych bets

In [2]:
for year in range(2021, 2024):
    file_name=f"Raw data/{year}.xlsx"
    variable_name=f"bets_{year}"
    globals()[variable_name] = pd.read_excel(file_name)
    print(f"Loaded {variable_name} from {file_name}")

Loaded bets_2021 from Raw data/2021.xlsx
Loaded bets_2022 from Raw data/2022.xlsx
Loaded bets_2023 from Raw data/2023.xlsx


In [3]:
for i in range(2021,2024):
    variable_name=f"bets_{i}"
    globals()[variable_name].loc[globals()[variable_name]["Tournament"] == "Adelaide International 1", "Location"] = "Adelaide 1"
    globals()[variable_name].loc[globals()[variable_name]["Tournament"] == "Adelaide International 2", "Location"] = "Adelaide 2"
    globals()[variable_name]["Location"] = globals()[variable_name]["Location"].replace({'Dubai ': 'Dubai', 'Belgrade ': 'Belgrade', 'Napoli':'Naples', 'Montreal': 'Toronto'})
    globals()[variable_name] = globals()[variable_name][~globals()[variable_name]['Location'].isin(['Turin'])]
    globals()[variable_name]['Loser'] = globals()[variable_name]['Loser'].replace({"Varillas J. P.": "Varillas J.P.", "Tseng C. H.": "Tseng C.H."})
    globals()[variable_name]['Winner'] = globals()[variable_name]['Winner'].replace({"Varillas J. P.": "Varillas J.P.", "Tseng C. H.": "Tseng C.H."})
    globals()[variable_name] = globals()[variable_name][~globals()[variable_name]['Tournament'].isin(['United Cup', 'Tour Finals', 'NextGen Finals','Tokyo Olympics','Atp Cup', 'Great Ocean Road Open', 'Murray River Open',
     'Laver Cup', 'Melbourne Summer Set'])]
    globals()[variable_name].loc[globals()[variable_name]['Tournament']== 'BNP Paribas Masters', 'Location']='Paris 2'
    globals()[variable_name].loc[globals()[variable_name]['Tournament']== 'Belgrade Open', 'Location']='Belgrade 2'
    
    

In [4]:
bets_2022.loc[(bets_2022['Tournament'] == 'Atlanta Open') & (bets_2022['Winner'] == 'Martin A.'), 'Winner'] = 'Martin An.'
bets_2022.loc[(bets_2022['Tournament'] == 'Atlanta Open') & (bets_2022['Loser'] == 'Martin A.'), 'Loser'] = 'Martin An.'

In [5]:
for i in range(2021,2024):
    variable_name=f"bets_{i}"
    globals()[variable_name].rename(columns={'Location': 'tourney_location'}, inplace=True)
    globals()[variable_name].reset_index(drop=True, inplace=True)

In [5]:
for i in range(2021,2024):
    variable_name=f"bets_{i}"
    print(f"rok{i} {globals()[variable_name].columns}")

rok2021 Index(['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5',
       'L5', 'Wsets', 'Lsets', 'Comment', 'B365W', 'B365L', 'PSW', 'PSL',
       'MaxW', 'MaxL', 'AvgW', 'AvgL'],
      dtype='object')
rok2022 Index(['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5',
       'L5', 'Wsets', 'Lsets', 'Comment', 'B365W', 'B365L', 'PSW', 'PSL',
       'MaxW', 'MaxL', 'AvgW', 'AvgL'],
      dtype='object')
rok2023 Index(['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5',
       'L5', 'Wsets', '

## ZBIORY RÓŻNIĄ SIĘ BUKMACHERAMI, DLATEGO ZBIERAMY WSZYSTKIE UNIKATOWE NAZWY BUKMACHEROW I UPEWNIAMY SIE, ŻE KAŻDY ZBIÓR MA KAŻDEGO BUKMACHERA POPRZEZ DODANIE KOLUMNY, WARTOŚCI DLA NOWO DODANEJ KOLUMNY SĄ NAN

In [6]:
for i in range(2021, 2024):
    variable_name = f"bets_{i}"
    variable_name_bookmakers = f"bets_{i}_bookmakers"
    globals()[variable_name_bookmakers] = globals()[variable_name].copy()
    
    columns_to_drop = ['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
                       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
                       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
                       'Comment']
    
    globals()[variable_name_bookmakers] = globals()[variable_name_bookmakers].drop(columns=columns_to_drop)
    

## ZEBRANIE WSZYSTKICH UNIKALNYCH NAZW BUKMACHERÓW

In [7]:
unique_columns = set()
for i in range(2021, 2024):
    variable_name_bookmakers = f"bets_{i}_bookmakers"
    if variable_name_bookmakers in globals() and isinstance(globals()[variable_name_bookmakers], pd.DataFrame):
        unique_columns.update(globals()[variable_name_bookmakers].columns)


## UZUPEŁNIENIE BRAKUJĄCYCH KOLUMN W bets_{rok} I USTAWIENIE WARTOŚCI NA NAN

In [8]:
for i in range(2021, 2024):
    variable_name = f"bets_{i}"
    if variable_name in globals() and isinstance(globals()[variable_name], pd.DataFrame):
        missing_columns = unique_columns.difference(globals()[variable_name].columns)
        for column in missing_columns:
            globals()[variable_name][column] = None

In [9]:
bets_2021.columns

Index(['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5',
       'L5', 'Wsets', 'Lsets', 'Comment', 'B365W', 'B365L', 'PSW', 'PSL',
       'MaxW', 'MaxL', 'AvgW', 'AvgL'],
      dtype='object')

## PONIŻSZA FUNKCJA NIE MA ŻADNEGO OUTPUTU, TZN ŻE WSZYSTKO JEST W PORZĄDKU I DANE MOŻNA ZMERGOWAĆ

In [10]:
for i in range(2021, 2024):
    variable_name=f"bets_{i}"
    columns=set(globals()[variable_name].columns)
    for j in range(i+1, 2024):
        variable_name_2=f"bets_{j}"
        if columns != set(globals()[variable_name_2].columns):
            print(f"Rok: {i} niezgodny z rokiem {j}")

## Wczytanie danych github

In [6]:
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    file_name = f"Raw data/atp_matches_{i}.csv"
    globals()[variable_name] = pd.read_csv(file_name)
    print(f"Loaded {variable_name} from {file_name}")

Loaded github_2021 from Raw data/atp_matches_2021.csv
Loaded github_2022 from Raw data/atp_matches_2022.csv
Loaded github_2023 from Raw data/atp_matches_2023.csv


## wszystkie zbiory mają jednakowe kolumny :))

In [12]:
k=0
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    columns=set(globals()[variable_name].columns)
    for j in range(i+1, 2024):
        variable_name_2=f"github_{j}"
        if columns != set(globals()[variable_name_2].columns):
            print(f"Rok: {i} niezgodny z rokiem {j}")
            k+=1
print(k)

0


### Zgodnie z założeniami usuwamy Finals, NextGen Finals, United Cup oraz mecze Davis Cup

In [7]:
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    globals()[variable_name]=globals()[variable_name][~globals()[variable_name]['tourney_name'].str.contains('Davis Cup', na=False)]
    globals()[variable_name] = globals()[variable_name][~globals()[variable_name]['tourney_name'].isin(['United Cup', 'Tour Finals', 'NextGen Finals','Tokyo Olympics','Atp Cup', 'Great Ocean Road Open', 'Murray River Open',
  'Laver Cup','Melbourne Summer Set', 'Melbourne'])]
    globals()[variable_name]["tourney_name"] = globals()[variable_name]["tourney_name"].replace({'Belgrade ': 'Belgrade'})
    
    

### W tej ramce kolumna 'tourney_name' to dla większości turniejów lokalizacja turnieju a nie jego nazwa, a potrzebujemy żeby nazwa turnieju była taka jak w ramce bets. Dla turniejów, dla których 'tourney_name' to nie lokalizacja, ręcznie wpisujemy lokalizację

In [8]:
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    globals()[variable_name].rename(columns={'tourney_name': 'tourney_location'}, inplace=True)
    globals()[variable_name]['tourney_location'] = globals()[variable_name]['tourney_location'].replace({"Australian Open": "Melbourne", "Indian Wells Masters": "Indian Wells", "Miami Masters": "Miami", "Monte Carlo Masters": "Monte Carlo", "Madrid Masters": "Madrid", "Rome Masters": "Rome", "Roland Garros": "Paris", "s Hertogenbosch": "'s-Hertogenbosch", "Queen's Club": "Queens Club", "Wimbledon": "London", "Canada Masters": "Toronto", "Cincinnati Masters": "Cincinnati", "Us Open": "New York", "Astana": "Nur-Sultan", "Shanghai Masters": "Shanghai", "Paris Masters": "Paris 2", "Rio De Janeiro": "Rio de Janeiro"})

## Po sprawdzeniu danych historycznych wykryliśmy błędy 

In [9]:
github_2023['loser_name'] = github_2023['loser_name'].replace('Eduardo Nava', 'Emilio Nava')
github_2023['loser_id'] = github_2023['loser_id'].replace(124013, 207182)
github_2021['loser_name'] = github_2021['loser_name'].replace('Eduardo Nava', 'Emilio Nava')
github_2021['loser_id'] = github_2021['loser_id'].replace(124013, 207182)
#github_2022['loser_name']=github_2022['loser_name'].replace('Andres Martin', 'Andrej Martin')
#github_2022['loser_id']=github_2022['loser_id'].replace(211346,105413)


### Sprawdzamy, że utworzona przez nas kolumnna 'tourney_location' odpowiada kolumnie 'tourney_location' w ramce bets

In [10]:
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    print(globals()[variable_name][~globals()[variable_name]['tourney_location'].isin(globals()[variable_name_2]['tourney_location'])]['tourney_location'].unique())
    print(globals()[variable_name_2][~globals()[variable_name_2]['tourney_location'].isin(globals()[variable_name]['tourney_location'])]['tourney_location'].unique())
    

[]
[]
[]
[]
[]
[]


### Dzięki temu możemy zamienić wartości kolumny 'tourney_level' na wartości kolumny 'Series' z ramki bets. Dziwne określenie rangi turnieju za pomoca literek zmieniamy na standardowe rangi ATP250, ATP500 etc. Ręcznie musimy jedynie poprawić turniej Masters 1000 w Paryżu, gdyż również w Paryżu odbywa się turniej rangi Grand Slam.

In [17]:
# for year in range(2021, 2024):
#     github_name = f"github_{year}"
#     bets_name = f"bets_{year}"
    
#     github_df = globals()[github_name]
#     bets_df = globals()[bets_name]
    
#     merged_df = github_df.merge(bets_df[['tourney_location', 'Series']], on='tourney_location', how='left')
#     github_df['tourney_level'] = merged_df['Series']
#     globals()[github_name] = github_df

## to działa 

In [11]:
for i in range(2021, 2024):
    github_name = f"github_{i}"
    bets_name = f"bets_{i}"
    
    github_df = globals()[github_name]
    bets_df = globals()[bets_name]
    df2=bets_df.drop_duplicates(subset='tourney_location', keep='first')

    merged_df = pd.merge(github_df, df2[['tourney_location', 'Series']], on='tourney_location', how='left')
    merged_df['tourney_level'] = merged_df['Series']
    merged_df = merged_df.drop(columns=['Series'])
    globals()[github_name]=merged_df
    

### Możemy teraz stworzyć w obu ramkach klucz 'tournament_id', który jest połaczeniem 'tourney_location' oraz 'tourney_level' i jednoznacznie określa turniej

In [12]:
for i in range(2021,2024):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    globals()[variable_name]['tournament_id']=globals()[variable_name]['tourney_location'].astype(str)+ '_' + globals()[variable_name]['tourney_level'].astype(str)+ '_' + str(i)
    globals()[variable_name_2]['tournament_id']=globals()[variable_name_2]['tourney_location'].astype(str)+ '_' + globals()[variable_name_2]['Series'].astype(str)+ '_' + str(i)
    

### Sprawdzamy czy w obu ramkach mamy te same klucze turnieju

In [13]:
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    print(globals()[variable_name][~globals()[variable_name]['tournament_id'].isin(globals()[variable_name_2]['tournament_id'])]['tournament_id'].unique())
    print(globals()[variable_name_2][~globals()[variable_name_2]['tournament_id'].isin(globals()[variable_name]['tournament_id'])]['tournament_id'].unique())

[]
[]
[]
[]
[]
[]


### Potrzebujemy mieć takie same nazwy zawodników w obu ramkach, aby dodać do ramki bets id zawodnika z ramki github. Aktualnie w ramce bets nazwa zawodnika jest w formacie typu Djokovic N. a w ramce github w formacie typu Novak Djokovic, dlatego musimy stworzyć nazwy zawodników w skróconym w formacie w ramce github.

In [14]:
def transform_name(name):
    name_parts = name.split()
    first_name = name_parts[0]
    last_name = ' '.join(name_parts[1:])
    return f"{last_name} {first_name[0]}."

In [15]:
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    globals()[variable_name]['shortened_winner_name']=globals()[variable_name]['winner_name'].apply(transform_name)
    globals()[variable_name]['shortened_loser_name']=globals()[variable_name]['loser_name'].apply(transform_name)

### Sprawdzamy czy skrócona nazwa zawodnika jednoznacznie określa zawodnika

In [19]:
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    print(globals()[variable_name]['shortened_winner_name'].nunique() == globals()[variable_name]['winner_id'].nunique())
    print(globals()[variable_name]['shortened_loser_name'].nunique() == globals()[variable_name]['loser_id'].nunique())

True
True
True
True
True
True


## Zobaczmy, którzy zawodnicy nie są jednoznacznie określeni przez skróconą nazwę

In [20]:
non_unique_losers={}
for i in range(2021,2024):
    variable_name=f"github_{i}"
    loser_groups = globals()[variable_name].groupby('shortened_loser_name')['loser_id'].nunique()
    non_unique_loser_names = loser_groups[loser_groups > 1].index.tolist()
    non_unique_losers[i] = globals()[variable_name][globals()[variable_name]['shortened_loser_name'].isin(non_unique_loser_names)][['shortened_loser_name', 'loser_id']].drop_duplicates()

for i in range(2021,2024):
    if not non_unique_losers[i].empty:
        print(f"Lata {i} - Przegrani z niejednoznacznymi skróconymi nazwami:")
        print(non_unique_losers[i])

105413- Martin andrej

## W celu rozróżnienia zawodników do skrótu imienia jednego z nich dopsujemy drugą literkę imienia

In [18]:
for i in range(2021, 2024):
   variable_name=f"github_{i}"
   globals()[variable_name].loc[globals()[variable_name]['loser_id'] == 124013, 'shortened_loser_name'] = 'Nava Ed.'
for i in range(2021, 2023):
   variable_name=f"github_{i}"
   globals()[variable_name].loc[globals()[variable_name]['loser_id'] == 211346, 'shortened_loser_name'] = 'Martin An.'
   globals()[variable_name].loc[globals()[variable_name]['winner_id'] == 211346, 'shortened_winner_name'] = 'Martin An.'

### Możemy teraz przypisać do zawodników w ramce bets id zawodnika pozyskane z ramki github. Sprawdzamy czy do jakiegoś zawodnika nie zostało przypisany id.

In [21]:
github_2023['shortened_loser_name'] = github_2023['shortened_loser_name'].replace({"Meligeni Alves F.":"Meligeni Rodrigues F","Arnaud Bailly G.":"Bailly G.","Sung Nam J.":"Nam J.S.","Chan Hong S.":"Hong S.","Fa Rodriguez Taverna S.":"Rodriguez Taverna S.","Pucinelli De Almeida M.":"Pucinelli de Almeida M.","Alejandro Hernandez Serrano J.":"Hernandez A.","Marcel Stebe C.":"Stebe C.M.","Martin del Potro J.":"Del Potro J.M.","Marco Moroni G.":"Moroni G.M.","Tsonga J.":"Tsonga J.W.","Ignacio Londero J.":"Londero J.I.","Pablo Varillas J.":"Varillas J. P.","Pablo Ficovich J.":"Ficovich J.P.","C.H. Tseng":"Tseng C. H.","Oconnell C.": "O Connell C.", "Elahi Galan D.": "Galan D.E.", "Auger Aliassime F.": "Auger-Aliassime F.", "Woo Kwon S.": "Kwon S.W.", "Barrios Vera T.": "Barrios M.", "Yunchaokete B.": "Bu Y.", "Manuel Cerundolo J.": "Cerundolo J.M.", "Martin Etcheverry T.": "Etcheverry T.", "Hugues Herbert P.": "Herbert P.H.", "Hsiou Hsu Y.": "Hsu Y.", "Andrea Huesler M.": "Huesler M.A.", "Kuznetsov A.": "Kuznetsov An.", "Son Kwiatkowski T.": "Kwiatkowski T.S.", "Li Z.": "Li Zh.", "Hsin Tseng C.": "Tseng C. H.", "Kumar Mukund S.": "Mukund S.", "Ramos A.": "Ramos-Vinolas A.", "J Wolf J.": "Wolf J.J.", "Zhang Z.": "Zhang Zh.", "Pablo Varillas J.": "Varillas J. P.", "Lennard Struff J.": "Struff J.L.", "Lin Wu T.": "Wu T.L.", "Hans Rehberg M.": "Rehberg M.", "Mpetshi Perricard G.": "Mpetshi G.", "Agustin Tirante T.": "Tirante T.A.", "Alberto Olivieri G.": "Olivieri G.", "Nicolae Madaras D.": "Madaras D.", "Cong Mo Y.": "Mo Y."})
github_2022['shortened_loser_name'] = github_2022['shortened_loser_name'].replace({"Meligeni Alves F.":"Meligeni Rodrigues F","Arnaud Bailly G.":"Bailly G.","Sung Nam J.":"Nam J.S.","Chan Hong S.":"Hong S.","Fa Rodriguez Taverna S.":"Rodriguez Taverna S.","Pucinelli De Almeida M.":"Pucinelli de Almeida M.","Alejandro Hernandez Serrano J.":"Hernandez A.","Marcel Stebe C.":"Stebe C.M.","Martin del Potro J.":"Del Potro J.M.","Marco Moroni G.":"Moroni G.M.","Tsonga J.":"Tsonga J.W.","Ignacio Londero J.":"Londero J.I.","Pablo Varillas J.":"Varillas J. P.","Pablo Ficovich J.":"Ficovich J.P.","C.H. Tseng":"Tseng C. H.","Oconnell C.": "O Connell C.", "Elahi Galan D.": "Galan D.E.", "Auger Aliassime F.": "Auger-Aliassime F.", "Woo Kwon S.": "Kwon S.W.", "Barrios Vera T.": "Barrios M.", "Yunchaokete B.": "Bu Y.", "Manuel Cerundolo J.": "Cerundolo J.M.", "Martin Etcheverry T.": "Etcheverry T.", "Hugues Herbert P.": "Herbert P.H.", "Hsiou Hsu Y.": "Hsu Y.", "Andrea Huesler M.": "Huesler M.A.", "Kuznetsov A.": "Kuznetsov An.", "Son Kwiatkowski T.": "Kwiatkowski T.S.", "Li Z.": "Li Zh.", "Hsin Tseng C.": "Tseng C. H.", "Kumar Mukund S.": "Mukund S.", "Ramos A.": "Ramos-Vinolas A.", "J Wolf J.": "Wolf J.J.", "Zhang Z.": "Zhang Zh.", "Pablo Varillas J.": "Varillas J. P.", "Lennard Struff J.": "Struff J.L.", "Lin Wu T.": "Wu T.L.", "Hans Rehberg M.": "Rehberg M.", "Mpetshi Perricard G.": "Mpetshi G.", "Agustin Tirante T.": "Tirante T.A.", "Alberto Olivieri G.": "Olivieri G.", "Nicolae Madaras D.": "Madaras D.", "Cong Mo Y.": "Mo Y."})
github_2021['shortened_loser_name'] = github_2021['shortened_loser_name'].replace({"Patrick Smith J.":"Smith J.P.","Shannan Zayid M.":"Zayid M.","Hsun Lu Y.":"Lu Y.","Aragone J.":"Aragone J.C.","Meligeni Alves F.":"Meligeni Rodrigues F","Arnaud Bailly G.":"Bailly G.","Sung Nam J.":"Nam J.S.","Chan Hong S.":"Hong S.","Fa Rodriguez Taverna S.":"Rodriguez Taverna S.","Pucinelli De Almeida M.":"Pucinelli de Almeida M.","Alejandro Hernandez Serrano J.":"Hernandez A.","Marcel Stebe C.":"Stebe C.M.","Martin del Potro J.":"Del Potro J.M.","Marco Moroni G.":"Moroni G.M.","Tsonga J.":"Tsonga J.W.","Ignacio Londero J.":"Londero J.I.","Pablo Varillas J.":"Varillas J. P.","Pablo Ficovich J.":"Ficovich J.P.","C.H. Tseng":"Tseng C. H.","Oconnell C.": "O Connell C.", "Elahi Galan D.": "Galan D.E.", "Auger Aliassime F.": "Auger-Aliassime F.", "Woo Kwon S.": "Kwon S.W.", "Barrios Vera T.": "Barrios M.", "Yunchaokete B.": "Bu Y.", "Manuel Cerundolo J.": "Cerundolo J.M.", "Martin Etcheverry T.": "Etcheverry T.", "Hugues Herbert P.": "Herbert P.H.", "Hsiou Hsu Y.": "Hsu Y.", "Andrea Huesler M.": "Huesler M.A.", "Kuznetsov A.": "Kuznetsov An.", "Son Kwiatkowski T.": "Kwiatkowski T.S.", "Li Z.": "Li Zh.", "Hsin Tseng C.": "Tseng C. H.", "Kumar Mukund S.": "Mukund S.", "Ramos A.": "Ramos-Vinolas A.", "J Wolf J.": "Wolf J.J.", "Zhang Z.": "Zhang Zh.", "Pablo Varillas J.": "Varillas J. P.", "Lennard Struff J.": "Struff J.L.", "Lin Wu T.": "Wu T.L.", "Hans Rehberg M.": "Rehberg M.", "Mpetshi Perricard G.": "Mpetshi G.", "Agustin Tirante T.": "Tirante T.A.", "Alberto Olivieri G.": "Olivieri G.", "Nicolae Madaras D.": "Madaras D.", "Cong Mo Y.": "Mo Y."})

In [22]:
github_2023['shortened_winner_name'] = github_2023['shortened_winner_name'].replace({"Varillas J.P.":"Varillas J. P.","Meligeni Alves F.":"Meligeni Rodrigues F","Arnaud Bailly G.":"Bailly G.","Sung Nam J.":"Nam J.S.","Chan Hong S.":"Hong S.","Fa Rodriguez Taverna S.":"Rodriguez Taverna S.","Pucinelli De Almeida M.":"Pucinelli de Almeida M.","Alejandro Hernandez Serrano J.":"Hernandez A.","Marcel Stebe C.":"Stebe C.M.","Martin del Potro J.":"Del Potro J.M.","Marco Moroni G.":"Moroni G.M.","Tsonga J.":"Tsonga J.W.","Ignacio Londero J.":"Londero J.I.","Pablo Varillas J.":"Varillas J. P.","Pablo Ficovich J.":"Ficovich J.P.","C.H. Tseng":"Tseng C. H.","Oconnell C.": "O Connell C.", "Elahi Galan D.": "Galan D.E.", "Auger Aliassime F.": "Auger-Aliassime F.", "Woo Kwon S.": "Kwon S.W.", "Barrios Vera T.": "Barrios M.", "Yunchaokete B.": "Bu Y.", "Manuel Cerundolo J.": "Cerundolo J.M.", "Martin Etcheverry T.": "Etcheverry T.", "Hugues Herbert P.": "Herbert P.H.", "Hsiou Hsu Y.": "Hsu Y.", "Andrea Huesler M.": "Huesler M.A.", "Kuznetsov A.": "Kuznetsov An.", "Son Kwiatkowski T.": "Kwiatkowski T.S.", "Li Z.": "Li Zh.", "Hsin Tseng C.": "Tseng C. H.", "Kumar Mukund S.": "Mukund S.", "Ramos A.": "Ramos-Vinolas A.", "J Wolf J.": "Wolf J.J.", "Zhang Z.": "Zhang Zh.", "Pablo Varillas J.": "Varillas J. P.", "Lennard Struff J.": "Struff J.L.", "Lin Wu T.": "Wu T.L.", "Hans Rehberg M.": "Rehberg M.", "Mpetshi Perricard G.": "Mpetshi G.", "Agustin Tirante T.": "Tirante T.A.", "Alberto Olivieri G.": "Olivieri G.", "Nicolae Madaras D.": "Madaras D.", "Cong Mo Y.": "Mo Y."})
github_2022['shortened_winner_name'] = github_2022['shortened_winner_name'].replace({"Varillas J.P.":"Varillas J. P.","Meligeni Alves F.":"Meligeni Rodrigues F","Arnaud Bailly G.":"Bailly G.","Sung Nam J.":"Nam J.S.","Chan Hong S.":"Hong S.","Fa Rodriguez Taverna S.":"Rodriguez Taverna S.","Pucinelli De Almeida M.":"Pucinelli de Almeida M.","Alejandro Hernandez Serrano J.":"Hernandez A.","Marcel Stebe C.":"Stebe C.M.","Martin del Potro J.":"Del Potro J.M.","Marco Moroni G.":"Moroni G.M.","Tsonga J.":"Tsonga J.W.","Ignacio Londero J.":"Londero J.I.","Pablo Varillas J.":"Varillas J. P.","Pablo Ficovich J.":"Ficovich J.P.","C.H. Tseng":"Tseng C. H.","Arnaud Bailly G.":"Bailly G.","Sung Nam J.":"Nam J.S.","Chan Hong S.":"Hong S.","Fa Rodriguez Taverna S.":"Rodriguez Taverna S.","Pucinelli De Almeida M.":"Pucinelli de Almeida M.","Alejandro Hernandez Serrano J.":"Hernandez A.","Marcel Stebe C.":"Stebe C.M.","Martin del Potro J.":"Del Potro J.M.","Marco Moroni G.":"Moroni G.M.","Tsonga J.":"Tsonga J.W.","Ignacio Londero J.":"Londero J.I.","Pablo Varillas J.":"Varillas J. P.","Pablo Ficovich J.":"Ficovich J.P.","C.H. Tseng":"Tseng C. H.","Oconnell C.": "O Connell C.", "Elahi Galan D.": "Galan D.E.", "Auger Aliassime F.": "Auger-Aliassime F.", "Woo Kwon S.": "Kwon S.W.", "Barrios Vera T.": "Barrios M.", "Yunchaokete B.": "Bu Y.", "Manuel Cerundolo J.": "Cerundolo J.M.", "Martin Etcheverry T.": "Etcheverry T.", "Hugues Herbert P.": "Herbert P.H.", "Hsiou Hsu Y.": "Hsu Y.", "Andrea Huesler M.": "Huesler M.A.", "Kuznetsov A.": "Kuznetsov An.", "Son Kwiatkowski T.": "Kwiatkowski T.S.", "Li Z.": "Li Zh.", "Hsin Tseng C.": "Tseng C. H.", "Kumar Mukund S.": "Mukund S.", "Ramos A.": "Ramos-Vinolas A.", "J Wolf J.": "Wolf J.J.", "Zhang Z.": "Zhang Zh.", "Pablo Varillas J.": "Varillas J. P.", "Lennard Struff J.": "Struff J.L.", "Lin Wu T.": "Wu T.L.", "Hans Rehberg M.": "Rehberg M.", "Mpetshi Perricard G.": "Mpetshi G.", "Agustin Tirante T.": "Tirante T.A.", "Alberto Olivieri G.": "Olivieri G.", "Nicolae Madaras D.": "Madaras D.", "Cong Mo Y.": "Mo Y."})
github_2021['shortened_winner_name'] = github_2021['shortened_winner_name'].replace({"Varillas J.P.":"Varillas J. P.","Patrick Smith J.":"Smith J.P.","Shannan Zayid M.":"Zayid M.","Hsun Lu Y.":"Lu Y.","Aragone J.":"Aragone J.C.","Meligeni Alves F.":"Meligeni Rodrigues F","Arnaud Bailly G.":"Bailly G.","Sung Nam J.":"Nam J.S.","Chan Hong S.":"Hong S.","Fa Rodriguez Taverna S.":"Rodriguez Taverna S.","Pucinelli De Almeida M.":"Pucinelli de Almeida M.","Alejandro Hernandez Serrano J.":"Hernandez A.","Marcel Stebe C.":"Stebe C.M.","Martin del Potro J.":"Del Potro J.M.","Marco Moroni G.":"Moroni G.M.","Tsonga J.":"Tsonga J.W.","Ignacio Londero J.":"Londero J.I.","Pablo Varillas J.":"Varillas J. P.","Pablo Ficovich J.":"Ficovich J.P.","C.H. Tseng":"Tseng C. H.","Oconnell C.": "O Connell C.", "Elahi Galan D.": "Galan D.E.", "Auger Aliassime F.": "Auger-Aliassime F.", "Woo Kwon S.": "Kwon S.W.", "Barrios Vera T.": "Barrios M.", "Yunchaokete B.": "Bu Y.", "Manuel Cerundolo J.": "Cerundolo J.M.", "Martin Etcheverry T.": "Etcheverry T.", "Hugues Herbert P.": "Herbert P.H.", "Hsiou Hsu Y.": "Hsu Y.", "Andrea Huesler M.": "Huesler M.A.", "Kuznetsov A.": "Kuznetsov An.", "Son Kwiatkowski T.": "Kwiatkowski T.S.", "Li Z.": "Li Zh.", "Hsin Tseng C.": "Tseng C. H.", "Kumar Mukund S.": "Mukund S.", "Ramos A.": "Ramos-Vinolas A.", "J Wolf J.": "Wolf J.J.", "Zhang Z.": "Zhang Zh.", "Pablo Varillas J.": "Varillas J. P.", "Lennard Struff J.": "Struff J.L.", "Lin Wu T.": "Wu T.L.", "Hans Rehberg M.": "Rehberg M.", "Mpetshi Perricard G.": "Mpetshi G.", "Agustin Tirante T.": "Tirante T.A.", "Alberto Olivieri G.": "Olivieri G.", "Nicolae Madaras D.": "Madaras D.", "Cong Mo Y.": "Mo Y."})

In [23]:
bets_2022['Loser']=bets_2022['Loser'].replace({"Tseng C.H.":"Tseng C. H."})
bets_2022['Loser']=bets_2022['Loser'].replace({"Varillas J.P.":"Varillas J. P."})
bets_2023['Loser']=bets_2023['Loser'].replace({"Tseng C.H.":"Tseng C. H."})
bets_2023['Loser']=bets_2023['Loser'].replace({"Varillas J.P.":"Varillas J. P."})
bets_2023['Loser']=bets_2023['Loser'].replace({"Meligeni Alves F.":"Meligeni Rodrigues F"})
bets_2021['Loser']=bets_2021['Loser'].replace({"Varillas J.P.":"Varillas J. P."})

In [24]:
bets_2022['Winner']=bets_2022['Winner'].replace({"Tseng C.H.":"Tseng C. H."})
bets_2022['Winner']=bets_2022['Winner'].replace({"Varillas J.P.":"Varillas J. P."})
bets_2023['Winner']=bets_2023['Winner'].replace({"Tseng C.H.":"Tseng C. H."})
bets_2023['Winner']=bets_2023['Winner'].replace({"Varillas J.P.":"Varillas J. P."})
bets_2023['Winner']=bets_2023['Winner'].replace({"Meligeni Alves F.":"Meligeni Rodrigues F"})
bets_2021['Winner']=bets_2021['Winner'].replace({"Varillas J.P.":"Varillas J. P."})

## Po poprawkach każdy zawodnik w ramce bets ma już id

In [25]:
for i in range(2021,2024):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    map_player_name_player_id = globals()[variable_name].groupby('shortened_loser_name', as_index=False)[['shortened_loser_name', 'loser_id']].first()
    mapping = dict(zip(map_player_name_player_id['shortened_loser_name'], map_player_name_player_id['loser_id']))
    globals()[variable_name_2]['loser_id'] = globals()[variable_name_2]['Loser'].map(mapping)
    

In [26]:
bets_2023.loc[bets_2023['loser_id'].isna(), 'Loser'].unique()

array([], dtype=object)

In [27]:
bets_2022.loc[bets_2022['loser_id'].isna(), 'Loser'].unique()

array([], dtype=object)

In [28]:
bets_2021.loc[bets_2021['loser_id'].isna(), 'Loser'].unique()

array([], dtype=object)

In [29]:
for i in range(2021,2024):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    map_player_name_player_id = globals()[variable_name].groupby('shortened_winner_name', as_index=False)[['shortened_winner_name', 'winner_id']].first()
    mapping = dict(zip(map_player_name_player_id['shortened_winner_name'], map_player_name_player_id['winner_id']))
    globals()[variable_name_2]['winner_id'] = globals()[variable_name_2]['Winner'].map(mapping)

In [30]:
bets_2023.loc[bets_2023['winner_id'].isna(), 'Winner'].unique()

array([], dtype=object)

In [31]:
bets_2022.loc[bets_2022['winner_id'].isna(), 'Winner'].unique()

array([], dtype=object)

In [32]:
bets_2021.loc[bets_2021['winner_id'].isna(), 'Winner'].unique()

array([], dtype=object)

### Skoro mamy już wspólne id zawodnika, oraz id turnieju to możemy w obu ramkach zdefiniować wspólne id meczu. Sprawdzamy czy id meczu jest unikalne dla każdego wiersza w ramkach

In [33]:
for i in range(2021, 2024):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    globals()[variable_name]['match_id'] = globals()[variable_name]['tournament_id'].astype(str) + '_' +  globals()[variable_name]['winner_id'].astype(str) + '_' + globals()[variable_name]['loser_id'].astype(str)
    globals()[variable_name_2]['match_id'] = globals()[variable_name_2]['tournament_id'].astype(str) + '_' +  globals()[variable_name_2]['winner_id'].astype(str) + '_' + globals()[variable_name_2]['loser_id'].astype(str)

In [34]:
for i in range(2021,2024):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    print(globals()[variable_name]['match_id'].nunique() == len(globals()[variable_name]))
    print(globals()[variable_name_2]['match_id'].nunique() == len(globals()[variable_name_2]))


True
True
True
True
True
True


### W bets_2023 jest źle wpisywany wynik meczu Bu Y. vs Kecmanovic M. w pierwszej rundzie turnieju w Shanghaju., wpisane jest zwycięstwo Kecmanovica a przegrał on ten mecz. Taka sama sytaacja wystąpiła w meczu 1/8 finału pomiędzy Kuzmanov D. a Carbales Baena R. Musimy zamienić ze soba wszystkie kolumny zawierajace wartości dedykowane zwycięzcy/przegranemu

In [35]:
def swap_winner_loser_bets(match_ids):
    winner_cols = [col for col in bets_2023.columns if col.startswith('W') or col.endswith('W') or col == 'Winner']
    loser_cols = [col for col in bets_2023.columns if col.startswith('L') or col.endswith('L') or col == 'Loser']
    condition = bets_2023['match_id'].isin(match_ids)
    
    if not bets_2023[condition].empty:
        for w_col, l_col in zip(winner_cols, loser_cols):
            bets_2023.loc[condition, w_col], bets_2023.loc[condition, l_col] = bets_2023.loc[condition, l_col].values, bets_2023.loc[condition, w_col].values
    else:
        print(f"No match found with match_id {match_ids}")
    
    return bets_2023

In [36]:
bets_2023 = swap_winner_loser_bets(["Marrakech_ATP250_2023_106220_106148", "Shanghai_Masters 1000_2023_200175_207352"])
bets_2023['match_id'] = bets_2023['match_id'].replace({"Marrakech_ATP250_2023_106220_106148": "Marrakech_ATP250_2023_106148_106220", "Shanghai_Masters 1000_2023_200175_207352": "Shanghai_Masters 1000_2023_207352_200175"})

### Sprawdźmy jeszcze czy sa jakieś mecze, które sa tylko w jednej ramce.

In [37]:
for i in range(2021,2024):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    print(globals()[variable_name_2][~globals()[variable_name_2]['match_id'].isin(globals()[variable_name]['match_id'])]['match_id'].unique())
    print(globals()[variable_name][~globals()[variable_name]['match_id'].isin(globals()[variable_name_2]['match_id'])]['match_id'].unique())
    

[]
[]
[]
[]
[]
[]


In [38]:
folder_path = 'D:/tennis_data/Tennis/Processed data/'
bets_2023.to_csv(folder_path +'bets2023.csv', index=False)
bets_2022.to_csv(folder_path +'bets2022.csv', index=False)
bets_2021.to_csv(folder_path +'bets2021.csv', index=False)
github_2023.to_csv(folder_path +'github2023.csv', index=False)
github_2022.to_csv(folder_path +'github2022.csv', index=False)
github_2021.to_csv(folder_path +'github2021.csv', index=False)